In [1]:
import dill
import numpy as np
import tensorflow as tf

from agents.capm_agent import CAPM_Agent
from trading_env.environment import TradingEnv

In [2]:
from matplotlib import pyplot as plt
from scipy.optimize import LinearConstraint

In [3]:
yf_file = "./data/archive_data/yf_data.dill"
with open(yf_file,'rb') as dill_file:
    yf_df = dill.load(dill_file)
    

In [4]:
# Risk free rate
r = 1e-4

In [5]:
symbols = yf_df['Symbol'].unique()
stocks = []
for symbol in symbols:
    df = yf_df[yf_df['Symbol'] == symbol]
    stocks.append(df.iloc[:,2:6].to_numpy())
stocks = np.array(stocks)

In [6]:
# Batch the data
div = list(map(int,np.linspace(0,5284,51)))
batched_data = np.split(stocks,div[1:-1],axis=1)

# Segment batches into training batches, 1 validation batch, 1 testing batch 
training_stock_data = batched_data[:-2]
validation_stock_data = batched_data[-2]
test_stock_data = batched_data[-1]


In [7]:
training_stock_data[0].shape

(21, 105, 4)

In [10]:
env = TradingEnv(training_stock_data[0])
obs = env.reset()

In [8]:
capm_agent = CAPM_Agent(21)

In [11]:
capm_agent.act(obs)

array([1.07273687e-16, 0.00000000e+00, 2.17202590e-01, 1.45250564e-01,
       0.00000000e+00, 2.75295038e-03, 0.00000000e+00, 6.02742363e-17,
       6.93796943e-17, 2.83258991e-01, 1.46950136e-17, 0.00000000e+00,
       0.00000000e+00, 7.29093687e-17, 8.86724180e-02, 9.66632624e-17,
       5.86467730e-02, 0.00000000e+00, 2.04215714e-01, 0.00000000e+00,
       2.94117792e-17])

In [ ]:
K = ((obs[:-1,1:,-1] - obs[:-1,:-1,-1])/obs[:-1,:-1,-1])

In [ ]:
Sigma = np.cov(K)
sigma = np.diagonal(Sigma)

In [ ]:
mu = np.mean(K,axis=1)

In [ ]:
plt.scatter(sigma,mu)

In [ ]:
Ones = np.ones(21)
I = np.eye(21)
M = np.vstack((Ones,I))

In [ ]:
eq_constraint = LinearConstraint(Ones, [1], [1])
ineq_constraint = LinearConstraint(I,21*[0],21*[1])

In [ ]:
linear_constraint = LinearConstraint(M, [1]+21*[0], 22*[1])

In [ ]:
from scipy.optimize import minimize

In [ ]:
def market_portfolio(w):
    num = np.dot(w,mu) - r
    denom = np.sqrt(np.matmul(np.matmul(w,Sigma),w))
    return - num / denom 

In [ ]:
w = Ones / Ones.sum()

res = minimize(market_portfolio, w, constraints=[eq_constraint,ineq_constraint])

In [ ]:
res['x']

In [ ]:
SigmaInv = np.linalg.inv(Sigma)
Ones = np.ones(21)

W = np.matmul((mu-r*Ones),SigmaInv)
w = W / W.sum()

In [ ]:
market_portfolio(w)

In [ ]:
env = TradingEnv(training_stock_data[0])
obs = env.reset()
done = False
last_raw_action = tf.zeros((1,n_stocks+1))
while not done:
    raw_action = agent.act(obs,last_raw_action) 
    obs, reward, done, _ = env.step(agent.model.softmax_layer(raw_action))
    last_raw_action = raw_action 
print('Untrained portfolio end value:')
print(env.portfolio_value_hist[-1])